In [27]:
import numpy as np # 계산을 위해 행렬과 같은 다양한 수학 도구를 사용하기 위한 라이브러리
import os
import pandas as pd
import random

#Machine Learning Library
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle           

#Plotting Libraries
import seaborn as sn; sn.set(font_scale=1.4)
import matplotlib.pyplot as plt             

#openCV
import cv2                                 

#Tensor Flow & Keras
import tensorflow as tf    
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

#Train & Test Data Split
from sklearn.model_selection import train_test_split

#Garbage Collector
import gc





from keras.callbacks import EarlyStopping, ModelCheckpoint


In [29]:
from PIL import Image # Python Image Library  (파이썬을 이용해서 쉽게 이미지 프로세싱을 할 수 있게 주는 라이브러리)

import numpy as np # 계산을 위해 행렬과 같은 다양한 수학 도구를 사용하기 위한 라이브러리
import os # OS는 operating system의 약자로, 운영체제를 의미 파이썬에 기본적으로 내장된 모듈의 이름
           # os.path 모듈은 파일명과 파일경로에 대한 유용한 함수들을 많이 제공함
            # 경로의 이미지를 모두 뽑아내기 위한 os.listdir 사용예제
        
import glob # 재귀적으로 현재 폴더의 모든 하위폴더까지 탐색하여 확장자가 jpg인 파일을 출력한다.
            # 특정 파일만 출력하기
#openCV
import cv2   
    


from sklearn.model_selection import train_test_split

caltech_dir = "dataset/train" # fire 데이터 폴더 경로
categories = ["neutral","angry","surprise","smile","sad"]  # 각 dataset 폴더이름
nb_classes = len(categories)

image_w = 48
image_h = 48

X = []
y = []

for idx, cat in enumerate(categories):
    # 입력값에 따른 결과값
    # 분류 클래스 길이 만큼 선언한 뒤
    # 속하는 클래스에 참 거짓(1,0) 표현
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    

    image_dir = caltech_dir + "/" + cat #이미지가 있는 디렉터리 경로 설정
    files = glob.glob(image_dir+"/*.jpg") # 이미지 파일 가져오기
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files): # 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
        view_img = cv2.imread(f, cv2.IMREAD_COLOR)
        view_img = cv2.cvtColor(view_img, cv2.COLOR_BGR2GRAY)
        view_img = cv2.resize(view_img,(image_w, image_h))
#      아마도 cv2는 numpy 형태로 반환함
        X.append(view_img)
        y.append(label)


# 정규화
X = np.array(X)/255.0
y = np.array(y)

X = X.reshape(-1, 48, 48, 1)

# #dataset을 train data와 validation data를 나누어주는 함수
X_train, X_test, y_train, y_test = train_test_split(X, y)

print("ok", len(y))

neutral  파일 길이 :  6916
angry  파일 길이 :  5485
surprise  파일 길이 :  4300
smile  파일 길이 :  9889
sad  파일 길이 :  6439
ok 33029


In [30]:
print(X_train.shape)
print(X_test.shape)

(24771, 48, 48, 1)
(8258, 48, 48, 1)


In [36]:
print(view_img.shape)
print(X.shape)

(48, 48)
(30219, 48, 48, 1)


In [31]:
model = Sequential()
input_shape = (48,48,1)
model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
    
#   학습 과정 설정
#   손실 함수 및 최적화 방법을 정의
#   다중 분류 손실함수로 적합 categorical_crossentropy (미니 배치 확률적 경사 하강법)
#   최적화 식 adma : 학습률을 줄여나가고 속도를 계산하여 학습의 갱신강도를 적응적으로 조정해나가는 방법
#   (Gradient와 Learning Rate를 수정해줌)
#   metrics : 평가기준 (accuracy 시 평가기준은 정확도)  제대로 학습되고 있는 지 확인하는 용도
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = './model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
        
    # 모델 저장 경로
model_path = model_dir + '/fire_detection_images_classification.model'

#  훈련하는 동안 체크포인트를 저장하는 것 (에포크마다 학습된 가중치를 파일로 저장할 수 있다.)
#  모델을 재사용하거나 훈련을 이어서 할 수 있다.
#   ModelCheckpoint의 속성으로 verbose는 해당 함수의 진행 사항의 출력 여부 
#   save_best_only는 모델의 정확도가 최고값을 갱신했을 때만 저장하도록 하는 옵션
#   val_loss 값이 개선되었을때 호출된다.
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)

#    오버피팅(과적합)을 피하기 위한 기술    
#  ‘EarlyStopping’이라는 함수를 사용하며 더 이상 개선의 여지가 없을 때 학습을 종료시키는 콜백함수    
#   개선이 없다고 바로 종료하지 않고 개선이 없는 에포크를 얼마나 기다려 줄 것인 가를 지정
#   만약 10이라고 지정하면 개선이 없는 에포크가 10번째 지속될 경우 학습을 종료.
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
# 학습 시작
#  X : 입력 데이터
#  Y : 결과(Label 값) 데이터
#  batch_size : 한 번에 학습할 때 사용하는 데이터 개수
#  epochs : 학습 데이터 반복 횟수 (전체 훈련 데이터 반복 횟수)
#  callbacks : 학습과 검증 과정에서 적용할 호출 리스트
#  validation_data : 학습 도중 학습의 정확도를 검증할 데이터
history = model.fit(X_train, y_train, batch_size=64, epochs=200, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])


Train on 24771 samples, validate on 8258 samples
Epoch 1/200
19392/24771 [======================>.......] - ETA: 2:52 - loss: 1.4586 - accuracy: 0.3949

In [ ]:
import cv2
import sys
# 입력 파일 지정하기

caltech_dir = "nota/train/img" # fire 데이터 폴더 경로
files = glob.glob(caltech_dir+"/*.jpg") # 이미지 파일 가져오기




# 캐스케이드 파일의 경로 지정하기 --- (※1)
cascade_file = "haarcascade_frontalface_default.xml"


for i, f in enumerate(files): # 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
    view_img = cv2.imread(f, cv2.COLOR_BGR2GRAY)
    face_extractor(image)




# 이미지 읽어 들이기 --- (※2)
image = cv2.imread(image_file)
# 그레이스케일로 변환하기
image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 얼굴 인식 특징 파일 읽어 들이기 --- (※3)
cascade = cv2.CascadeClassifier(cascade_file)
# 얼굴 인식 실행하기
face_list = cascade.detectMultiScale(image_gs,
    scaleFactor=1.1,
    minNeighbors=1,
    minSize=(150,150))
if len(face_list) > 0:
    # 인식한 부분 표시하기 --- (※4)
    print(face_list)
    color = (0, 0, 255)
    for face in face_list:
        x,y,w,h = face
        cv2.rectangle(image, (x,y), (x+w, y+h), color, thickness=8)
    # 파일로 출력하기 --- (※5)
    cv2.imwrite("facedetect-output.PNG", image)
else:
    print("no face")

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#전체 사진에서 얼굴 부위만 잘라 리턴
def face_extractor(img):
    global count
    
    #흑백처리 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #얼굴 찾기 
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    #찾은 얼굴이 없으면 None으로 리턴 
    if faces is():
        return None
    #얼굴들이 있으면 
    for(x,y,w,h) in faces:
        #해당 얼굴 크기만큼 cropped_face에 잘라 넣기 
        #근데... 얼굴이 2개 이상 감지되면??
        #가장 마지막의 얼굴만 남을 듯
        cropped_face = img[y:y+h, x:x+w]
        face = cv2.resize(cropped_face,(200,200))
        cv2.imwrite('facetest/face_test'+str(count)+'.jpg',face)
        count+=1
    #cropped_face 리턴 

In [5]:
# 노타 데이터 분류해서 폴더 이동
from pandas.io.parsers import read_csv #엑셀 데이터를 읽어오기 위해 필요
import shutil

data_csv = read_csv('data/train_labels.csv', sep=',') # '를 토큰으로 데이터를 분리해서 담을수 있도록 
train = np.array(data_csv)

src = 'dataset/nota/data/'
dir = 'dataset/nota/train/'
for f in train:
    filename = f[0]
    try:
        shutil.move(src+filename, dir + f[3] +'/'+filename)
    except:
        print(f[0])


100.VD34944438_w640.jpg
101.EliKiseopUKISS.jpg
103.1131271546.jpg
103.crop.JPG
103.image_crop.JPG
107.crop.JPG
11.happy-asian-family-stock-photo-2726146.jpg
11.happy-asian-family-stock-photo-2726146.jpg
11.happy-asian-family-stock-photo-2726146.jpg
111.Asian-girls-shopping-515.jpg
111.Asian-girls-shopping-515.jpg
111.Asian-girls-shopping-515.jpg
111.Asian-girls-shopping-515.jpg
111.Asian-girls-shopping-515.jpg
111.Asian-girls-shopping-515.jpg
111.Asian-girls-shopping-515.jpg
114.crop.JPG
114.crop.JPG
119.IE002370253_PHT.jpg
119.IE002370253_PHT.jpg
12.crop.JPG
12.crop.JPG
12.maxresdefault.jpg
120.1133637106.jpg
123.focused_178419342-stock-photo-happy-asian-family-jumping-hoppity.jpg
123.focused_178419342-stock-photo-happy-asian-family-jumping-hoppity.jpg
124.maxresdefault.jpg
13.helping-children-deal-with-angry.jpg
131.234_crop.JPG
132.2022403.jpg
132.2022403.jpg
134.84575477.jpg
134.x1080-u-0.jpg
136.94b5682b46cc4ddc967774e8815c4356.jpg
137.Rithika-Kodithuwakku-and-Swarna-Mallawarachcc

351.fbl-asia-c2-final-al-hilal-wanderers_fn1985.jpg
354.VD12441121_w640.jpg
358.903224498.jpg
361.42599_620.jpg
361.42599_620.jpg
361.42599_620.jpg
372.HappyEnding4-00019.jpg
372.HappyEnding4-00019.jpg
373.crop.JPG
376.asian-little-girl-take-picture-crying-little-boy_43289-43.jpg
382.Pictureday.jpg
382.Pictureday.jpg
382.Pictureday.jpg
382.Pictureday.jpg
382.Pictureday.jpg
382.Pictureday.jpg
382.Pictureday.jpg
382.Pictureday.jpg
39.1.jpg
39.8f516a5958de65a189bfd5855bc5b87e.jpg
390.crop4.JPG
392.854010618.jpg
392.854010618.jpg
392.VD89131005_w640.jpg
394.crop.JPG
395.aotp-asian-buddy-quiz-index.jpg
395.aotp-asian-buddy-quiz-index.jpg
395.aotp-asian-buddy-quiz-index.jpg
407.portrait-happy-asian-mother-use-nose-touch-newborn-cute-little-baby-nose-white-bed-home_78681-15.jpg
411.asian_crop.JPG
413.10566921_f520.jpg
413.10566921_f520.jpg
413.10566921_f520.jpg
413.123903908-angry-asian-couple-lover-in-white-t-shirt-and-grey-background-.jpg
413.happy-asian-wife-angry-loser-husband_39688-1532.

In [ ]:
import glob # 재귀적으로 현재 폴더의 모든 하위폴더까지 탐색하여 확장자가 jpg인 파일을 출력한다.
            # 특정 파일만 출력하기

caltech_dir = "dataset/nota/train" # fire 데이터 폴더 경로
class_dir = ['angry','neutral', 'sad', 'smile', 'surprise']
files = glob.glob(caltech_dir+"/*.jpg") # 이미지 파일 가져오기
cascade_file = "haarcascade_frontalface_default.xml" # 캐스케이드 파일의 경로 지정하기 --- (※1)


for i, f in enumerate(files): # 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
    view_img = cv2.imread(f, cv2.COLOR_BGR2GRAY)
    face_extractor(image)

In [19]:
import glob # 재귀적으로 현재 폴더의 모든 하위폴더까지 탐색하여 확장자가 jpg인 파일을 출력한다.
            # 특정 파일만 출력하기

global count
count=0
caltech_dir = "dataset/nota/train/" # fire 데이터 폴더 경로
class_dir = ['angry','neutral', 'sad', 'smile', 'surprise']



for cdir in class_dir:
    files = glob.glob(caltech_dir+cdir+"/*.jpg") # 이미지 파일 가져오기

    for i, f in enumerate(files): # 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
        view_img = cv2.imread(f, cv2.COLOR_BGR2GRAY)
        face_extractor(view_img, cdir)

In [18]:


#전체 사진에서 얼굴 부위만 잘라 리턴
def face_extractor(img, cdir):
    global count
    path = 'dataset/nota/train/crop/'
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    #얼굴 찾기 
    faces = face_classifier.detectMultiScale(img,1.3,5)
    #찾은 얼굴이 없으면 None으로 리턴 
    if faces is():
        return None
    #얼굴들이 있으면 
    for(x,y,w,h) in faces:
        #해당 얼굴 크기만큼 cropped_face에 잘라 넣기 
        #근데... 얼굴이 2개 이상 감지되면??
        #가장 마지막의 얼굴만 남을 듯
        cropped_face = img[y:y+h, x:x+w]
        face = cv2.resize(cropped_face,(48,48))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(path+cdir+'/'+str(count)+'.jpg',face)
        count+=1
    #cropped_face 리턴 